# Reproducing Parameter Estimations

This notebook demonstrates the steps to reproduce parameter estimations for enzyme cascade reactions. The workflow includes data loading, calibration, activity calculation, and parameter fitting.

In [2]:
# Import required libraries
import os
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

from data_hadler import*
from parameter_estimator import*

In [3]:
BASE_PATH = r"C:\Users\berger\Documents\Projekts\enzyme-cascade-analysis\Fehlerfortpflanzunganalyse"

calibration_data = pd.read_csv(os.path.join(BASE_PATH, 'Data', 'NADH_Kalibriergerade.csv'))
calibration_slope = calculate_calibration(calibration_data)

r1_path = os.path.join(BASE_PATH, 'Data', 'Reaction1')
r1_nad_data = pd.read_csv(os.path.join(r1_path, 'r_1_NAD_PD_500mM.csv'), header=None)
r1_pd_data = pd.read_csv(os.path.join(r1_path, 'r1_PD_NAD_5mM.csv'), header=None)


In [4]:


reac_1_data = {
    "r1_nad": r1_nad_data,
    "r1_pd": r1_pd_data
}


reac1_activity_param = {
    "r1_nad": {
        "Vf_well": 10.0,
        "Vf_prod": 1.0,
        "c_prod": 2.2108
    },
    "r1_pd": {
        "Vf_well": 10.0,
        "Vf_prod": 1.0,
        "c_prod": 2.2108
    }
}

constants_dict = {
    "r1_nad_const": 5.0,    # NAD konstant wenn PD variiert
    "r1_pd_const": 500.0   # PD konstant wenn NAD variiert
}

processed_data = get_rates_and_concentrations(
    reac_1_data, 
    calibration_slope, 
    reac1_activity_param
)
# Fix: get_rates function needs 3 parameters, not 2
reac_1_rates  = create_reaction_rates_dict(processed_data)

reac_1_concentrations = create_concentrations_dict(processed_data, constants_dict)


Verarbeite Reaktion: r1_nad
Well 1 (Konz: 0.0 mM): R² = 0.092 - nicht linear genug
Well 2 (Konz: 0.0 mM): R² = 0.097 - nicht linear genug
Well 3: 1.00 mM → Aktivität: 0.022898 U/mg
Well 4: 1.00 mM → Aktivität: 0.021928 U/mg
Well 5: 2.00 mM → Aktivität: 0.031797 U/mg
Well 6: 2.00 mM → Aktivität: 0.034679 U/mg
Well 7: 3.00 mM → Aktivität: 0.038776 U/mg
Well 8: 3.00 mM → Aktivität: 0.039649 U/mg
Well 9: 4.00 mM → Aktivität: 0.042043 U/mg
Well 10: 4.00 mM → Aktivität: 0.045990 U/mg
Well 11: 5.00 mM → Aktivität: 0.050625 U/mg
Well 12: 5.00 mM → Aktivität: 0.050075 U/mg
Well 13: 6.00 mM → Aktivität: 0.052413 U/mg
Well 14: 6.00 mM → Aktivität: 0.050983 U/mg
Well 15: 7.00 mM → Aktivität: 0.050328 U/mg
Well 16: 7.00 mM → Aktivität: 0.053309 U/mg
Erfolgreich 14 gültige Datenpunkte verarbeitet
✓ r1_nad: 14 gültige Datenpunkte
Verarbeite Reaktion: r1_pd
Well 1 (Konz: 0.0 mM): R² = 0.506 - nicht linear genug
Well 2 (Konz: 0.0 mM): R² = 0.382 - nicht linear genug
Well 3 (Konz: 25.0 mM): R² = 0.860 -

In [5]:

def two_substrat_michaelis_menten(concentration_data, Vmax, Km1, Km2): 
    """Zwei-Substrat Michaelis-Menten Gleichung"""
    S1_values, S2_values = concentration_data

    # Sicherstellen, dass es numpy arrays sind
    S1_values = np.asarray(S1_values)
    S2_values = np.asarray(S2_values)
    
    # Element-weise Berechnung für alle Datenpunkte
    rates = (Vmax * S1_values * S2_values) / ((Km1 + S1_values) * (Km2 + S2_values))
    
    return rates

reac_1_data_info = {
    "active_params": reac1_activity_param,
    "constants": constants_dict
}

reac_1_model_info = {
    "name": "two_substrat_michaelis_menten",
    "function": two_substrat_michaelis_menten,
    "param_names": ["Vmax", "Km_NAD", "Km_PD"],
    "param_units": ["U", "mM", "mM"],
    "substrate_keys": ["r1_nad_conc", "r1_pd_conc"],
    "initial_guess_func": lambda activities, substrate_data: [max(activities) if len(activities) > 0 else 1.0, 1.0, 1.0],
    "bounds_lower": [0, 0, 0],
    "bounds_upper": [np.inf, np.inf, np.inf],
    "description": "Zwei-Substrat Michaelis-Menten für Reaktion 1"
}

reac_1_parameters = estimate_parameters(reac_1_model_info, reac_1_data_info, reac_1_concentrations, reac_1_rates)

print("\n=== Reaktion 1 Parameter Schätzung ==="
        f"\nModell: {reac_1_model_info['description']}"
        f"\nErgebnis: {reac_1_parameters}"
        f"\nR²: {reac_1_parameters['r_squared']:.4f}"
        f"\nVmax: {reac_1_parameters['params'][0]:.4f} {reac_1_model_info['param_units'][0]}"
        f"\nKm1: {reac_1_parameters['params'][1]:.4f} {reac_1_model_info['param_units'][1]}"
        f"\nKm2: {reac_1_parameters['params'][2]:.4f} {reac_1_model_info['param_units'][2]}"
        f"\nVmax Fehler: {reac_1_parameters['param_errors'][0]:.4f} {reac_1_model_info['param_units'][0]}"
        f"\nKm1 Fehler: {reac_1_parameters['param_errors'][1]:.4f} {reac_1_model_info['param_units'][1]}"
        f"\nKm2 Fehler: {reac_1_parameters['param_errors'][2]:.4f} {reac_1_model_info['param_units'][2]}"
    )


Initial guess: [np.float64(0.05330916209607437), 1.0, 1.0]

=== Reaktion 1 Parameter Schätzung ===
Modell: Zwei-Substrat Michaelis-Menten für Reaktion 1
Ergebnis: {'success': True, 'params': array([7.74413089e-02, 1.93540939e+00, 1.01810991e+02]), 'param_errors': array([4.89151331e-03, 3.34588769e-01, 1.29390003e+01]), 'r_squared': np.float64(0.9359502024467375), 'model_name': 'two_substrat_michaelis_menten', 'description': 'Zwei-Substrat Michaelis-Menten für Reaktion 1'}
R²: 0.9360
Vmax: 0.0774 U
Km1: 1.9354 mM
Km2: 101.8110 mM
Vmax Fehler: 0.0049 U
Km1 Fehler: 0.3346 mM
Km2 Fehler: 12.9390 mM


In [6]:

noise_level = {
    "calibration": 0.01,
    "reaction": 0.01
}

monte_carlo_results = monte_carlo_simulation_r1( calibration_data, reac_1_data, reac_1_model_info, reac_1_data_info, noise_level, n_iterations=1000 )

if monte_carlo_results:
    print("\n" + "="*60)
    print("📊 MONTE CARLO ERGEBNISSE")
    print("="*60)
    print(f"✅ Erfolgreiche Iterationen: {monte_carlo_results['n_successful']}/{monte_carlo_results['n_total']}")
    print(f"📈 Erfolgsrate: {monte_carlo_results['success_rate']*100:.1f}%")
    print(f"📊 R² (Mittelwert): {monte_carlo_results['R_squared_mean']:.4f} ± {monte_carlo_results['R_squared_std']:.4f}")
    
    param_names = monte_carlo_results['param_names']
    param_units = reac_1_model_info['param_units']
    
    for i, param_name in enumerate(param_names):
        mean_key = f"{param_name}_mean"
        std_key = f"{param_name}_std"
        if mean_key in monte_carlo_results and std_key in monte_carlo_results:
            unit = param_units[i] if i < len(param_units) else ""
            print(f"🔸 {param_name}: {monte_carlo_results[mean_key]:.4f} ± {monte_carlo_results[std_key]:.4f} {unit}")
else:
    print("\n❌ Monte Carlo Simulation fehlgeschlagen!")


🔬 MONTE CARLO SIMULATION
Modell: Zwei-Substrat Michaelis-Menten für Reaktion 1
Iterationen: 1000
Kalibrierungs-Rauschen: 1.0%
Reaktions-Rauschen: 1.0%
🔄 Fortschritt:   5.0% (  50/1000) | ✅ Erfolg:  50 (100.0%)
🔄 Fortschritt:   5.0% (  50/1000) | ✅ Erfolg:  50 (100.0%)
🔄 Fortschritt:  10.0% ( 100/1000) | ✅ Erfolg: 100 (100.0%)
🔄 Fortschritt:  10.0% ( 100/1000) | ✅ Erfolg: 100 (100.0%)
🔄 Fortschritt:  15.0% ( 150/1000) | ✅ Erfolg: 150 (100.0%)
🔄 Fortschritt:  15.0% ( 150/1000) | ✅ Erfolg: 150 (100.0%)
🔄 Fortschritt:  20.0% ( 200/1000) | ✅ Erfolg: 200 (100.0%)
🔄 Fortschritt:  20.0% ( 200/1000) | ✅ Erfolg: 200 (100.0%)
🔄 Fortschritt:  25.0% ( 250/1000) | ✅ Erfolg: 250 (100.0%)
🔄 Fortschritt:  25.0% ( 250/1000) | ✅ Erfolg: 250 (100.0%)
🔄 Fortschritt:  30.0% ( 300/1000) | ✅ Erfolg: 300 (100.0%)
🔄 Fortschritt:  30.0% ( 300/1000) | ✅ Erfolg: 300 (100.0%)
🔄 Fortschritt:  35.0% ( 350/1000) | ✅ Erfolg: 350 (100.0%)
🔄 Fortschritt:  35.0% ( 350/1000) | ✅ Erfolg: 350 (100.0%)
🔄 Fortschritt:  40.0% 